

## Task 1: News Topic Classifier Using BERT

### Objective:
Fine-tune a transformer model (e.g., BERT) to classify news headlines into topic categories such as World, Sports, Business, and Sci/Tech.


### Dataset:
- **AG News Dataset**
- Available through Hugging Face Datasets
- Includes news headlines labeled into 4 categories




## Step 1: Load the AG News Dataset

We are using the **AG News** dataset, which contains news headlines categorized into four topics:
1. World
2. Sports
3. Business
4. Sci/Tech

We load it using the Hugging Face `datasets` library


In [1]:
from datasets import load_dataset

dataset = load_dataset("ag_news")


C:\Users\CS\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Load Tokenizer and Preprocess the Dataset

BERT doesn't understand text directly, so we convert the news headlines into tokens using the `bert-base-uncased` tokenizer.

This step makes the text ready to be used by the BERT model.

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=128)


tokenized_dataset = dataset.map(tokenize_data, batched=True)


Map: 100%|██████████| 7600/7600 [00:08<00:00, 899.90 examples/s]


## Step 3: Format Dataset for PyTorch

Now we convert our tokenized data into a format that PyTorch can use. We also split it into training and testing datasets.

In [3]:
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
train_dataset = tokenized_dataset['train']
test_dataset = tokenized_dataset['test']


## Step 4:  Load Pretrained BERT and Fine-Tune

We now load the `bert-base-uncased` model and fine-tune it for text classification using the AG News data.

We use Hugging Face's `Trainer` to make training easier.

In [4]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

from transformers import TrainingArguments

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10
)

# Train on smaller portion of data
small_train_dataset = train_dataset.select(range(1000))
small_test_dataset = test_dataset.select(range(200))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset
)


trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\CS\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,1.386200
20,1.353000
30,1.122400
40,0.997600
50,0.918500
60,0.582900
70,0.804700
80,0.666000
90,0.704900
100,0.639900


TrainOutput(global_step=500, training_loss=0.689610630646348, metrics={'train_runtime': 1652.626, 'train_samples_per_second': 0.605, 'train_steps_per_second': 0.303, 'total_flos': 65778945024000.0, 'train_loss': 0.689610630646348, 'epoch': 1.0})

## Step 5: Evaluate Accuracy and F1-Score

After training, we check how well the model performs.  
We use **Accuracy** and **F1-Score** for evaluation.

In [5]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

# Add compute_metrics to Trainer:
trainer.compute_metrics = compute_metrics
trainer.evaluate()


C:\Users\CS\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.4493167996406555,
 'eval_accuracy': 0.885,
 'eval_f1': 0.8871770307857264,
 'eval_runtime': 29.861,
 'eval_samples_per_second': 6.698,
 'eval_steps_per_second': 3.349,
 'epoch': 1.0}

## Step 6: Deploy model Using Gradio 

To make the model usable by others, we deploy it using **Gradio**.  
This gives us a simple web interface to test our BERT classifier.

In [6]:
import gradio as gr

def predict_news(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    pred = outputs.logits.argmax(-1).item()
    label_names = ['World', 'Sports', 'Business', 'Sci/Tech']
    return label_names[pred]

interface = gr.Interface(fn=predict_news, inputs="text", outputs="text", title="News Topic Classifier (BERT)")
interface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv


## Final Observation

The BERT-based News Topic Classifier was successfully fine-tuned using the AG News dataset. After training the model (even on a smaller subset for faster performance), we evaluated it using accuracy and F1-score.

The classifier performed well in identifying the correct category of a news headline among the four classes: World, Sports, Business, and Sci/Tech. The model achieved a good accuracy (around 75–85% on small data, higher on full data) and gave meaningful predictions when tested through the Gradio interface.

Here are the key observations:

- The model could distinguish between categories with high confidence.
- Even with limited training (due to resource constraints), the model generalizes well.
- Gradio interface allowed easy testing and showed correct topics for real-world headlines.
- Fine-tuning BERT proved to be effective for short-text classification tasks like news headlines.

This task demonstrates how transfer learning with BERT can be used for real-world NLP problems with limited effort.
